#Installing Packages

In [ ]:
pip install openai==0.28

: 

In [ ]:
pip install scikit-learn

In [ ]:
pip install PyMuPDF

In [ ]:
pip install PyPDF2

#Importing Libraries

In [ ]:
import re
import fitz  # PyMuPDF
import openai
import PyPDF2

#OpenAI API Key

In [ ]:
# Set up OpenAI API Key
openai.api_key = "OPEN AI API KEY"

Resume Path

In [ ]:
resume_path = "Resume.pdf"  # Replace with the actual resume file path

#Saving Candidate Info

In [ ]:

# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text("text") + "\n"
    return text

# Function to extract email using regex
def extract_email(text):
    email_pattern = r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}"
    email_matches = re.findall(email_pattern, text)
    return email_matches[0] if email_matches else "Not Found"

# Function to extract phone number using regex
def extract_phone(text):
    phone_pattern = r"\+?\d{1,3}?[-.\s]?\(?\d{2,4}?\)?[-.\s]?\d{3,5}[-.\s]?\d{4,6}"
    phone_matches = re.findall(phone_pattern, text)
    return phone_matches[0] if phone_matches else "Not Found"

# Function to extract only the candidate's name using OpenAI GPT
def extract_name(text):
    prompt = f"Extract only the full name of the candidate from the following resume text. Return only the name and nothing else:\n{text[:1000]}"
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "system", "content": "You are an expert at extracting names from resumes."},
                  {"role": "user", "content": prompt}]
    )
    return response["choices"][0]["message"]["content"].strip()

# Function to extract and save details
def extract_and_save_info(pdf_path, output_file="CDD_Info.txt"):
    text = extract_text_from_pdf(pdf_path)
    name = extract_name(text)
    phone = extract_phone(text)
    email = extract_email(text)

    # Save details to file
    with open(output_file, "w") as file:
        file.write(f"Name: {name}\n")
        file.write(f"Phone Number: {phone}\n")
        file.write(f"Email: {email}\n")

    print(f"Details saved successfully in {output_file}!")

# Example usage

extract_and_save_info(resume_path)


#Saving Questions

In [ ]:
import PyPDF2
import openai

def extract_text_from_pdf(pdf_path):
    """Extract text from a PDF file."""
    try:
        with open(pdf_path, 'rb') as pdf_file:
            reader = PyPDF2.PdfReader(pdf_file)
            text = ""
            for page in reader.pages:
                text += page.extract_text() or ""  # Handle None case
            return text.strip()
    except Exception as e:
        print(f"Error reading PDF: {e}")
        return ""

def read_job_description(txt_path):
    """Read job description from a text file."""
    try:
        with open(txt_path, "r", encoding="utf-8") as file:
            return file.read().strip()
    except Exception as e:
        print(f"Error reading job description file: {e}")
        return ""

def generate_interview_questions(job_description, resume):
    """Generate interview questions based on the provided job description and candidate's resume."""
    messages = [
        {"role": "system", "content": "You are an expert interviewer skilled in generating questions for job candidates."},
        {"role": "user", "content": f"Based on the following job description and resume, generate a list of potential interview questions:\n\nJob Description:\n{job_description}\n\nCandidate Resume:\n{resume}\n\nQuestions should cover technical skills, problem-solving, and role-specific knowledge."}
    ]

    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages,
            max_tokens=500,
            temperature=0.7
        )

        # Extract generated text
        questions = response['choices'][0]['message']['content'].strip()
        return questions.split("\n")  # Return questions as a list
    except Exception as e:
        print("Error:", e)
        return []

def save_questions_to_file(questions, filename="Questions.txt"):
    """Save the generated questions to a text file."""
    try:
        with open(filename, "w", encoding="utf-8") as file:
            for idx, question in enumerate(questions, start=1):
                file.write(f"{question}\n")
        print(f"Questions saved successfully in {filename}!")
    except Exception as e:
        print(f"Error writing to file: {e}")

In [ ]:
job_description_path = "/content/Director_JD.txt"  # Path to the job description text file

job_description = read_job_description(job_description_path)
resume_text = extract_text_from_pdf(resume_path)

if job_description and resume_text:
    questions = generate_interview_questions(job_description, resume_text)

    if questions:
        save_questions_to_file(questions)
    else:
        print("No questions generated.")
else:
    print("Failed to read job description or extract text from the resume.")


#Cutomized Calling

In [ ]:
import requests
import random
import openai

API_URL = "https://api.vapi.ai/assistant/{assistant_id}"
HEADERS = {
    "Authorization": "Bearer Token",
    "Content-Type": "application/json"
}

def read_candidate_name():
    """Read the candidate's name from CDD_Info.txt"""
    try:
        with open("CDD_Info.txt", "r") as file:
            return file.readline().strip()
    except Exception:
        return "Candidate"

def read_interview_questions():
    """Read and select 7 random questions from Questions.txt"""
    try:
        with open("Questions.txt", "r") as file:
            questions = [line.strip() for line in file.readlines() if line.strip()]
            return random.sample(questions, min(7, len(questions)))
    except Exception:
        return ["Can you tell me about yourself?", "Why do you want this role?", "What are your strengths and weaknesses?"]

candidate_name = read_candidate_name()
selected_questions = read_interview_questions()

# Prepare the AI Recruiter script
UPDATED_SCRIPT = {
    "model": {
        "model": "gpt-4o-mini",
        "provider": "openai",
        "temperature": 0.4,
        "messages": [
            {
                "role": "system",
                "content": f"""# Primary Task
You are now an AI Recruiter conducting a short pre-screening interview.
Greet the candidate by their name and confirm availability before proceeding.

# Behavior Guidelines
- Maintain a polite and professional tone.
- Ensure a smooth and engaging interview experience.
- Keep responses concise and structured.

# Role Instructions
1. Keep happy and cheerful tone.
2. Greet the candidate by their name.
3. Ask if they are available for a short interview.
4. If they agree:
 a. Ask them their curent location
 b. Ask them the expected salary per annum, if they tell it greater than 3 lakh rupees then laugh.
 c. ask them the following questions one by one:

  {chr(10).join([f'- {q}' for q in selected_questions])}

5. Acknowledge their responses and thank them for their time.

# Script
Greeting:
'Hello {candidate_name}! I a recruiter from Gappeo. How are you?'

(Wait for response)

If Available:
'Great! Let's get started.'

If Unavailable:
(ask them about another time at which they would be available, hear their response, thank them and then cut the call)

(Ask each question one by one)

Positive Conclusion:
'Thank you, {candidate_name}, for your time! We will review your responses and get back to you soon. Have a great day!'"""
            }
        ]
    }
}

response = requests.put(API_URL, headers=HEADERS, json=UPDATED_SCRIPT)

if response.status_code == 200:
    print("✅ AI Recruiter script updated successfully!")
else:
    print(f"❌ Error updating script: {response.json()}")


#Calling

In [ ]:
import requests

def read_candidate_details():
    """Read candidate's name, phone number, and email from CDD_Info.txt"""
    try:
        with open("CDD_Info.txt", "r") as file:
            lines = [line.strip() for line in file.readlines() if line.strip()]
            name = lines[0] if len(lines) > 0 else "Candidate"
            phone_number = lines[1] if len(lines) > 1 else "+91-0000000000"  # Default in case of missing value
            email = lines[2] if len(lines) > 2 else "candidate@example.com"
            return name, phone_number, email
    except Exception:
        return "Candidate", "+91-0000000000", "candidate@example.com"

# Read details
candidate_name, candidate_phone, candidate_email = read_candidate_details()

In [ ]:
import requests

# Create Call (POST /call)
response = requests.post(
  "https://api.vapi.ai/call",
  headers={
    "Authorization": "Bearer token",
    "Content-Type": "application/json"
  },
  json={
    "name": "name",
    "assistantId": "name",
    "phoneNumber": {
      "twilioAccountSid": "",
      "twilioAuthToken": "",
      "twilioPhoneNumber": ""
    },
    "customer": {
      "number": phone_number
    }
  },
)

# Parse response JSON
response_json = response.json()

# Extract the call ID
call_id = response_json.get("id")

print(response.json())
#print(call_id)

In [ ]:
import requests

# Get Call (GET /call/:id)
response = requests.get(
    f"https://api.vapi.ai/call/{call_id}",
    headers={
        "Authorization": "Bearer token"
    },
)

print(response.json())

# The last line seems misplaced; if it's a comment or a URL reference, clarify its purpose
# "https://api.vapi.ai/call/<Call_ID>"


In [ ]:
import requests

# Get Call (GET /call/:id)
response = requests.get(
    f"https://api.vapi.ai/call/{call_id}",
    headers={
        "Authorization": "Bearer token"
    },
)

# Extracting the 'transcript' from the JSON response
transcript = response.json().get("transcript", "")

# Save the transcript to a text file
with open("transcript.txt", "w") as file:
    file.write(transcript)

print("Transcript saved to transcript.txt")


In [ ]:
import re
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer

def extract_candidate_responses(conversation_text):
    """
    Extracts only the candidate's responses from a given conversation text.
    Assumes lines start with 'Recruiter:' or 'Candidate:'.
    """
    candidate_responses = []
    lines = conversation_text.split("\n")

    for line in lines:
        if line.startswith("Candidate:"):
            candidate_responses.append(line.replace("Candidate:", "").strip())

    return " ".join(candidate_responses)  # Merge all responses into a single text

def calculate_ats_score(candidate_text, jd_text):
    """
    Calculates ATS Score using TF-IDF similarity between candidate responses and job description.
    """
    vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = vectorizer.fit_transform([candidate_text, jd_text])
    similarity = (tfidf_matrix * tfidf_matrix.T).toarray()[0, 1]

    ats_score = round(similarity * 100, 2)  # Convert to percentage
    return ats_score

def categorize_candidate(score):
    """
    Categorizes candidate based on ATS score.
    """
    if score >= 80:
        return "Excellent"
    elif score >= 60:
        return "Good"
    elif score >= 40:
        return "Average"
    else:
        return "Weak"

# Load conversation and job description
conversation_file = "transcript.txt"  # Path to the conversation file
jd_file = "Sales_JD.txt"  # Path to the JD file

with open(conversation_file, "r", encoding="utf-8") as f:
    conversation_text = f.read()

with open(jd_file, "r", encoding="utf-8") as f:
    jd_text = f.read()

# Extract candidate responses
candidate_text = extract_candidate_responses(conversation_text)

# Calculate ATS Score
ats_score = calculate_ats_score(candidate_text, jd_text)

# Get candidate category
category = categorize_candidate(ats_score)

print(f"📊 Match Score: {ats_score}%")
print(f"🏅 Candidate Category: {category}")
